In [ ]:
import numpy as np
import pyaudio
import time
import matplotlib.pyplot as plt
from scipy import signal

# Parameters for audio capture
CHUNK = 1024         # Number of frames per buffer
RATE = 16000         # Sampling rate in Hz
WINDOW_DURATION = 2  # seconds
STEP_DURATION = 1    # seconds

# Derived parameters
window_size = int(RATE * WINDOW_DURATION)
step_size = int(RATE * STEP_DURATION)

# Initialize the audio stream
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)

# Initialize a buffer to store audio data
buffer = np.zeros(window_size, dtype=np.int16)

def compute_spectrogram(audio_segment, fs=RATE):
    """
    Compute the spectrogram of the given audio segment.
    """
    # Define parameters for spectrogram calculation
    nfft = 200  # Length of each window segment
    noverlap = 120  # Overlap between windows

    freqs, times, Sxx = signal.spectrogram(audio_segment, fs=fs, nperseg=nfft, noverlap=noverlap)
    return freqs, times, Sxx

def process_audio_segment(segment):
    """
    Process the audio segment by computing its spectrogram
    and passing it to your model.
    Return True if the trigger word is detected.
    """
    freqs, times, spectrogram = compute_spectrogram(segment)
    # For debugging: you could display the spectrogram with matplotlib
    # plt.pcolormesh(times, freqs, 10 * np.log10(spectrogram), shading='gouraud')
    # plt.ylabel('Frequency [Hz]')
    # plt.xlabel('Time [sec]')
    # plt.show()

    # TODO: Insert your model inference here using the spectrogram.
    # For now, we will simply return False as a placeholder.
    return False

print("Listening for the trigger word...")

try:
    while True:
        # Read new data from the stream
        new_data = stream.read(step_size)
        new_samples = np.frombuffer(new_data, dtype=np.int16)
        
        # Update the buffer: shift left and append new samples at the end
        buffer = np.roll(buffer, -len(new_samples))
        buffer[-len(new_samples):] = new_samples
        
        # Process the current buffer (or a part of it)
        if process_audio_segment(buffer):
            print("Trigger word detected!")
            # Insert further actions upon detection here.
        
        # Small sleep to avoid busy looping
        time.sleep(0.01)

except KeyboardInterrupt:
    print("Stopping...")
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


In [ ]:
import numpy as np
import pyaudio
import time
import matplotlib.pyplot as plt
from scipy import signal
import wave

# Parameters for audio capture
CHUNK = 1024         # Number of frames per buffer
RATE = 16000         # Sampling rate in Hz
WINDOW_DURATION = 0.8  # seconds
STEP_DURATION = 0.8    # seconds, set equal to WINDOW_DURATION for non-overlapping segments

# Derived parameters
window_size = int(RATE * WINDOW_DURATION)  # Total samples in 2 seconds
step_size = int(RATE * STEP_DURATION)        # Here, same as window_size

# Initialize the audio stream
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

def compute_spectrogram(audio_segment, fs=RATE):
    """
    Compute the spectrogram of the given audio segment.
    """
    nfft = 200      # Length of each window segment for FFT
    noverlap = 120  # Overlap between windows
    freqs, times, Sxx = signal.spectrogram(audio_segment, fs=fs, nperseg=nfft, noverlap=noverlap)
    return freqs, times, Sxx

def process_audio_segment(segment):
    """
    Process the audio segment by computing its spectrogram
    and passing it to your model.
    Return True if the trigger word is detected.
    """
    freqs, times, spectrogram = compute_spectrogram(segment)
    
    # For debugging: you can display the spectrogram with matplotlib
    # plt.pcolormesh(times, freqs, 10 * np.log10(spectrogram), shading='gouraud')
    # plt.ylabel('Frequency [Hz]')
    # plt.xlabel('Time [sec]')
    # plt.show()

    # TODO: Insert your model inference here using the spectrogram.
    # For now, we'll simply return False as a placeholder.
    return False

def save_audio_clip(audio_segment, filename, rate=RATE):
    """
    Save a numpy array of int16 audio samples as a WAV file.
    """
    wf = wave.open(filename, 'wb')
    wf.setnchannels(1)                          # Mono
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))  # 16-bit
    wf.setframerate(rate)
    wf.writeframes(audio_segment.tobytes())
    wf.close()

print("Listening for the trigger word and saving every 2 seconds of audio...")
counter = 0

try:
    while True:
        # Collect audio for a full 2-second segment
        frames = []
        num_frames = int(RATE / CHUNK * WINDOW_DURATION)  # How many CHUNKs in 2 seconds
        for _ in range(num_frames):
            data = stream.read(CHUNK)
            frames.append(data)
        # Concatenate the frames to form a 2-second segment
        audio_bytes = b''.join(frames)
        # Convert to a numpy array (int16) for processing or saving
        audio_np = np.frombuffer(audio_bytes, dtype=np.int16)
        
        # Save the 2-second audio segment to a WAV file
        counter += 1
        filename = f"rand_{counter}.wav"
        save_audio_clip(audio_np, filename, rate=RATE)
        print(f"Saved {filename}")
        
        # Process the segment (e.g., compute spectrogram and check for the trigger word)
        if process_audio_segment(audio_np):
            print("Trigger word detected!")
        
        # Optional short pause before recording the next segment
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Stopping...")
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


In [ ]:
import wave
import os

def chunk_wav_file(input_filename, chunk_duration=3, output_folder="chunks"):
    """
    Splits a WAV file into chunks of `chunk_duration` seconds and saves each chunk.
    
    Args:
        input_filename (str): Path to the input WAV file.
        chunk_duration (int, float): Duration of each chunk in seconds.
        output_folder (str): Folder where the chunk files will be saved.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    with wave.open(input_filename, 'rb') as wav_in:
        # Retrieve parameters from the input file
        nchannels = wav_in.getnchannels()
        sampwidth = wav_in.getsampwidth()
        framerate = wav_in.getframerate()
        nframes = wav_in.getnframes()
        comptype = wav_in.getcomptype()
        compname = wav_in.getcompname()
        
        # Calculate the number of frames per chunk
        chunk_frames = int(framerate * chunk_duration)
        total_chunks = (nframes + chunk_frames - 1) // chunk_frames  # Ceiling division
        
        print(f"Input file: {input_filename}")
        print(f"Channels: {nchannels}, Sample Width: {sampwidth} bytes, Framerate: {framerate} Hz")
        print(f"Total frames: {nframes}, Chunk duration: {chunk_duration} sec ({chunk_frames} frames per chunk)")
        print(f"Total chunks: {total_chunks}")
        
        chunk_count = 1358
        while True:
            frames = wav_in.readframes(chunk_frames)
            if not frames:
                break  # End of file reached
            chunk_count += 1
            output_filename = os.path.join(output_folder, f"chunk_{chunk_count:03d}.wav")
            
            with wave.open(output_filename, 'wb') as wav_out:
                wav_out.setnchannels(nchannels)
                wav_out.setsampwidth(sampwidth)
                wav_out.setframerate(framerate)
                wav_out.setcomptype(comptype, compname)
                wav_out.writeframes(frames)
            
            print(f"Saved chunk {chunk_count} as {output_filename}")

if __name__ == "__main__":
    input_wav = "WasherDryer_8.wav"  # Replace with your WAV file path
    chunk_wav_file(input_wav, chunk_duration=3)


In [ ]:
import librosa
import soundfile as sf

def resample_audio(input_path, output_path, target_sr=16000):
    # Load the audio file
    audio, sr = librosa.load(input_path, sr=None)  # Load with original sample rate
    print(f"Original Sample Rate: {sr}")  # Print original sample rate

    # Resample to 16kHz
    resampled_audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)

    # Save the resampled audio
    sf.write(output_path, resampled_audio, target_sr)
    print(f"Saved resampled file to {output_path}")


In [ ]:
import os
from glob import glob

input_folder = "Back_Ground"
output_folder = "bk"

os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

for file in glob(os.path.join(input_folder, "*.wav")):
    filename = os.path.basename(file)
    output_path = os.path.join(output_folder, filename)
    resample_audio(file, output_path)


In [ ]:
import os
import librosa
import soundfile as sf
from glob import glob

def batch_resample_librosa(input_folder, output_folder, target_sr=16000):
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    audio_files = glob(os.path.join(input_folder, "*.wav"))  # Get all WAV files
    for i, file in enumerate(audio_files):
        filename = os.path.basename(file)
        output_path = os.path.join(output_folder, filename)

        # Load original audio
        audio, sr = librosa.load(file, sr=None)
        
        # Resample
        resampled_audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        
        # Save resampled file
        sf.write(output_path, resampled_audio, target_sr)

        print(f"[{i+1}/{len(audio_files)}] Resampled {filename} from {sr}Hz to {target_sr}Hz")

# Example usage
batch_resample_librosa("data_set/RW", "data_set/rr")


In [ ]:
import os
from glob import glob

input_folder = "data_set/Back_Ground"

# Correct pattern: "*.wav" (not ".wav")
audio_files = glob(os.path.join(input_folder, "*.wav"))

print("Found files:", audio_files)


In [ ]:
print("Checking input folder:", os.path.exists('data_set/Back_Ground'))


In [1]:
import os
import wave

# Set the folder path
folder_path = "data_set/bk/"

# Define the required duration (in seconds)
required_duration = 3

# Iterate over all .wav files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Open the .wav file
            with wave.open(file_path, "r") as wav_file:
                # Get file properties
                frame_rate = wav_file.getframerate()  # Sampling rate
                num_frames = wav_file.getnframes()    # Total frames
                
                # Calculate duration in seconds
                duration = num_frames / float(frame_rate)

                # Delete file if not exactly 3 seconds
                if round(duration, 2) != required_duration:
                    os.remove(file_path)
                    print(f"Deleted: {filename} (Duration: {duration:.2f} sec)")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

print("Cleanup completed.")


Error processing chunk_010.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_010.wav'
Error processing chunk_100.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_100.wav'
Error processing chunk_1017.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_1017.wav'
Error processing chunk_1037.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_1037.wav'
Error processing chunk_1108.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_1108.wav'
Error processing chunk_1179.wav: [WinError 32] The process cannot access the file because it is being used by another process: 'data_set/bk/chunk_1179.wav'
Error processing chunk_1280.wav: [WinError 32] The process cannot ac